In [ ]:
import json
import numpy as np
import os
import pandas as pd
from pathlib import Path
from pprint import pprint

from powersimdata.scenario.scenario import Scenario

from data_processing.azure_blob_uploaders import BlobUtil


In [ ]:
loadzone2state = {
    "Bay Area": "California",
    "Central California": "California",
    "Northern California": "California",
    "Southeast California": "California",
    "Southwest California": "California",

    "Florida Panhandle": "Florida",
    "Florida North": "Florida",
    "Florida South": "Florida",

    "Georgia North": "Georgia",
    "Georgia South": "Georgia",

    "Chicago North Illinois": "Illinois",
    "Illinois Downstate": "Illinois",

    "Michigan Northern": "Michigan",
    "Michigan Southern": "Michigan",

    "Minnesota Northern": "Minnesota",
    "Minnesota Southern": "Minnesota",

    "Missouri East": "Missouri",
    "Missouri West": "Missouri",

    "Montana Eastern": "Montana",
    "Montana Western": "Montana",

    "New York City": "New York",
    "Upstate New York": "New York",

    "Western North Carolina": "North Carolina",

    "New Mexico Eastern": "New Mexico",
    "New Mexico Western": "New Mexico",

    "Ohio River": "Ohio",
    "Ohio Lake Erie": "Ohio",

    "Pennsylvania Eastern": "Pennsylvania",
    "Pennsylvania Western": "Pennsylvania",

    "Coast": "Texas",
    "East": "Texas",
    "East Texas": "Texas",
    "El Paso": "Texas", 
    "Far West": "Texas",
    "North": "Texas",
    "North Central": "Texas",
    "South": "Texas",
    "South Central": "Texas",
    "Texas Panhandle": "Texas",
    "West": "Texas",

    "Virginia Mountains": "Virginia",
    "Virginia Tidewater": "Virginia",
}

curtailmentResourceTypes = {
    'wind': 'wind_curtailment',
    'wind_offshore': 'wind_offshore_curtailment',
    'solar': 'solar_curtailment'
}


In [ ]:
interconnects = [
    'Eastern',
    'Western',
    'Texas',
]

locations = [
    'USA',
    'Eastern',
    'Western',
    'Texas',
    'Alabama',
    'Arizona',
    'Arkansas',
    'California',
    'Colorado',
    'Connecticut',
    'Delaware',
    'Florida',
    'Georgia',
    
    'Idaho',
    'Illinois',
    'Indiana',
    'Iowa',
    'Kansas',
    'Kentucky',
    'Louisiana',
    'Maine',
    'Maryland',
    'Massachusetts',
    'Michigan',
    'Minnesota',
    'Mississippi',
    'Missouri',
    'Montana',
    'Nebraska',
    'Nevada',
    'New Hampshire',
    'New Jersey',
    'New Mexico',
    'New York',
    'North Carolina',
    'North Dakota',
    'Ohio',
    'Oklahoma',
    'Oregon',
    'Pennsylvania',
    'Rhode Island',
    'South Carolina',
    'South Dakota',
    'Tennessee',
    'Utah',
    'Vermont',
    'Virginia',
    'Washington',
    'West Virginia',
    'Wisconsin',
    'Wyoming'
]


In [ ]:
# value_name is pg, curtailment, etc
# df can be any df where the index = date in UTC, columns = plant_ids, etc.. Column data must be numeric
def resample_by_day(df, id_name, value_name):
    # Roll up data by day
    df_by_day = df.resample('d').sum()

    # Turn timestamps into strings
    df_by_day.index = df_by_day.index.strftime('%Y-%m-%d')

    # Unpivot data so cols are now timestamp, id_name, value
    # If we're looking at PG there are now ~900k rows: 2.5k plants * 366 days 
    df_by_day = df_by_day.reset_index().melt(id_vars='UTC')

    df_by_day.columns = ['timestamp', id_name, value_name]

    return df_by_day


In [ ]:
# uses loadzone2state value
def add_plant_grid_info(grid, plant_by_day):
    plant_with_loc = plant_by_day.join(grid.plant[['zone_name', 'type', 'interconnect']], on='plant_id')
    plant_with_loc = plant_with_loc.rename(columns={ 'type': 'resource_type', 'zone_name': 'zone' })
    
    # Replace loadzone with state name
    plant_with_loc['zone'] = plant_with_loc['zone'].replace(loadzone2state)

    return plant_with_loc

In [ ]:
def get_curtailment_by_day(pg_by_day, wind, solar):
    pg_renewables = pg_by_day.loc[
        (pg_by_day['resource_type'] == 'solar') 
        | (pg_by_day['resource_type'] == 'wind') 
        | (pg_by_day['resource_type'] == 'wind_offshore')
    ]
    pg_renewables['resource_type'] = pg_renewables['resource_type'].replace(curtailmentResourceTypes)
    
    wind_by_day = resample_by_day(wind, 'plant_id', 'available')
    solar_by_day = resample_by_day(solar, 'plant_id', 'available')
    wind_and_solar = pd.concat([solar_by_day, wind_by_day])
    
    # Note: if a plant is in the wind / solar profile but not in the current scenario's grid, we discard it
    pg_renewables = pg_renewables.merge(wind_and_solar, how="left", on=['plant_id', 'timestamp'])
    
    pg_renewables['curtailment'] = pg_renewables['available'] - pg_renewables['pg']
    pg_renewables = pg_renewables.drop(columns=['pg', 'available'])
    
    return pg_renewables


In [ ]:
""" Format of output:
{
    Arizona: {
        coal: [
            { x: '2016-01-01', y: 1234 },
            { x: '2016-01-02', y: 1812 },
            ...
        ],
        wind: { 
            ... 
        }, ...
    },
    Washington: {
    ...
    }, ...
}

"""
# uses external locations value
def create_dict_of_plant_loc(plant_by_day, value_name):
    plant_dict = {}
    yearly_plant_dict = {}
    
    for location in locations:
        plant_dict[location] = {}
        yearly_plant_dict[location] = {}
        
        groupby_cols = [
            'timestamp', 
            'resource_type'
        ]
        if location == 'USA':
            plant_by_loc = plant_by_day.drop(columns=['plant_id', 'zone', 'interconnect'])
        elif location in interconnects:
            groupby_cols.append('interconnect')
            plant_by_loc = plant_by_day.loc[(plant_by_day['interconnect'] == location)]
            plant_by_loc = plant_by_loc.drop(columns=['plant_id', 'zone'])
        else:
            groupby_cols.append('zone')
            plant_by_loc = plant_by_day.loc[(plant_by_day['zone'] == location)]
            plant_by_loc = plant_by_loc.drop(columns=['plant_id'])

        # Sum values
        plant_by_loc = plant_by_loc.groupby(groupby_cols).sum()
        plant_by_loc = plant_by_loc.reset_index()
        # Change from MWh to GWh. Also adding 0 prevents -0.0
        plant_by_loc[value_name] = plant_by_loc[value_name].apply(lambda x: round(x/1000, 2) + 0) 

        # split by resource type and add each to pg
        resource_types = plant_by_loc.resource_type.unique()
        for resource_type in resource_types:
            resource_type_df = plant_by_loc.loc[(plant_by_loc['resource_type'] == resource_type)]
            resource_type_df = resource_type_df.rename(columns={ 'timestamp': 'x', value_name: 'y' })

            # example result: [ { x: '2016-01-23', y: 12345.67 }, ... ]
            plant_dict[location][resource_type] = resource_type_df[['x', 'y']].to_dict('records')
            yearly_plant_dict[location][resource_type] = round(resource_type_df['y'].sum(), 2)
    return plant_dict, yearly_plant_dict


In [ ]:
def upload_plant_dict(data_type, time, plant_loc_dict, scenario_id, blob_client, path, version):
    for location in plant_loc_dict:
        # Create local path if it doesn't exist
        local_save_path=f"{path}/{version}/{scenario_id}/{time}/{location}"
        Path(local_save_path).mkdir(parents=True, exist_ok=True)

        blob_path = f"{version}/{scenario_id}/{time}/{location}"

        blob_client.upload_dict_as_json_gzip(
            plant_loc_dict[location], 
            f"{local_save_path}/{data_type}.json.gzip",
            f"{blob_path}/{data_type}.json");
        

In [ ]:
def create_and_upload_plant_dicts(scenario_id, blob_client, path, version):
    print(f"\nStarting scenario {scenario_id}\n")
    s = Scenario(str(scenario_id))
    grid = s.state.get_grid()
    pg = s.state.get_pg()
    wind = s.state.get_wind()
    solar = s.state.get_solar()
    
    pg_by_day = resample_by_day(pg, 'plant_id', 'pg')
    pg_by_day = add_plant_grid_info(grid, pg_by_day)
    
    curtailment_by_day = get_curtailment_by_day(pg_by_day, wind, solar)
    
    print("\nGrouping data by location")
    daily_pg_loc_dict, yearly_pg_loc_dict = create_dict_of_plant_loc(pg_by_day, 'pg')
    daily_cur_loc_dict, yearly_cur_loc_dict = create_dict_of_plant_loc(curtailment_by_day, 'curtailment')
    
    print("\nUploading data\n")
    upload_plant_dict('pg', 'daily', daily_pg_loc_dict, scenario_id, blob_client, path, version)
    upload_plant_dict('pg', 'yearly', yearly_pg_loc_dict, scenario_id, blob_client, path, version)
    upload_plant_dict('curtailment', 'daily', daily_cur_loc_dict, scenario_id, blob_client, path, version)
    upload_plant_dict('curtailment', 'yearly', yearly_cur_loc_dict, scenario_id, blob_client, path, version)
    

# Run Everything

In [ ]:
%%time

version = 'v1' # increment me if changing format!
path = './tmp_nivo_data' # Don't include slash at the end
scenario_ids = [
    '1705', '1724', '1723', '1270', # macro grid
    '1152', '1151', '1149', # USA collab
    '1099', '1098', '1097', # USA independent
    '1206', '1205', '1204', # USA collab w/ storage
    '1244', '1245', '1257', # collab w/ renewables at retirements
    '1362', '1361', '1338', # offshore wind
    '824', '823'            # historical
]
conn_str = os.environ.get("BLOB_STORAGE_CONN_STR")
blob_client = BlobUtil(conn_str, 'grid-data')

for scenario_id in scenario_ids:
    create_and_upload_plant_dicts(scenario_id, blob_client, path, version)
    
print("\nfinished!!")